### Import packages

In [1]:
import itertools
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from utilities import cal_score, convert_types

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lightgbm/__init__.py:45: FutureWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS will be built by the Apple Clang compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you won't need to install the gcc compiler anymore.
Instead of that, you'll need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", FutureWarning)


### Columns

In [2]:
cols_num = ['txn_dt', 'total_floor', 'building_complete_dt', 'parking_area', 'parking_price', 'txn_floor', 'land_area', 'building_area', 'lat', 'lon', 'village_income_median', 'town_population', 'town_area', 'town_population_density', 'doc_rate', 'master_rate', 'bachelor_rate', 'jobschool_rate', 'highschool_rate', 'junior_rate', 'elementary_rate', 'born_rate', 'death_rate', 'marriage_rate', 'divorce_rate', 'N_50', 'N_500', 'N_1000', 'N_5000', 'N_10000', 'I_10', 'I_50', 'I_100', 'I_250', 'I_500', 'I_1000', 'I_5000', 'I_10000', 'I_MIN', 'II_10', 'II_50', 'II_100', 'II_250', 'II_500', 'II_1000', 'II_5000', 'II_10000', 'II_MIN', 'III_10', 'III_50', 'III_100', 'III_250', 'III_500', 'III_1000', 'III_5000', 'III_10000', 'III_MIN', 'IV_10', 'IV_50', 'IV_100', 'IV_250', 'IV_500', 'IV_1000', 'IV_5000', 'IV_10000', 'IV_MIN', 'V_10', 'V_50', 'V_100', 'V_250', 'V_500', 'V_1000', 'V_5000', 'V_10000', 'V_MIN', 'VI_50', 'VI_100', 'VI_250', 'VI_500', 'VI_1000', 'VI_5000', 'VI_10000', 'VI_MIN', 'VII_10', 'VII_50', 'VII_100', 'VII_250', 'VII_500', 'VII_1000', 'VII_5000', 'VII_10000', 'VII_MIN', 'VIII_10', 'VIII_50', 'VIII_100', 'VIII_250', 'VIII_500', 'VIII_1000', 'VIII_5000', 'VIII_10000', 'VIII_MIN', 'IX_10', 'IX_50', 'IX_100', 'IX_250', 'IX_500', 'IX_1000', 'IX_5000', 'IX_10000', 'IX_MIN', 'X_10', 'X_50', 'X_100', 'X_250', 'X_500', 'X_1000', 'X_5000', 'X_10000', 'X_MIN', 'XI_10', 'XI_50', 'XI_100', 'XI_250', 'XI_500', 'XI_1000', 'XI_5000', 'XI_10000', 'XI_MIN', 'XII_10', 'XII_50', 'XII_100', 'XII_250', 'XII_500', 'XII_1000', 'XII_5000', 'XII_10000', 'XII_MIN', 'XIII_10', 'XIII_50', 'XIII_100', 'XIII_250', 'XIII_500', 'XIII_1000', 'XIII_5000', 'XIII_10000', 'XIII_MIN', 'XIV_10', 'XIV_50', 'XIV_100', 'XIV_250', 'XIV_500', 'XIV_1000', 'XIV_5000', 'XIV_10000', 'XIV_MIN']
cols_cat = ['building_material', 'city', 'building_type', 'building_use', 'parking_way', 'town', 'village']
cols_bin = ['I_index_50', 'I_index_500', 'I_index_1000', 'II_index_50', 'II_index_500', 'II_index_1000', 'III_index_50', 'III_index_500', 'III_index_1000', 'IV_index_50', 'IV_index_500', 'IV_index_1000', 'IV_index_5000', 'V_index_50', 'V_index_500', 'V_index_1000', 'VI_10', 'VI_index_50', 'VI_index_500', 'VI_index_1000', 'VII_index_50', 'VII_index_500', 'VII_index_1000', 'VIII_index_50', 'VIII_index_500', 'VIII_index_1000', 'IX_index_50', 'IX_index_500', 'IX_index_1000', 'IX_index_5000', 'X_index_50', 'X_index_500', 'X_index_1000', 'XI_index_50', 'XI_index_500', 'XI_index_1000', 'XI_index_5000', 'XI_index_10000', 'XII_index_50', 'XII_index_500', 'XII_index_1000', 'XIII_index_50', 'XIII_index_500', 'XIII_index_1000', 'XIII_index_5000', 'XIII_index_10000', 'XIV_index_50', 'XIV_index_500', 'XIV_index_1000']
cols_feats = cols_num + cols_cat + cols_bin

col_target = 'total_price'
col_target_log1p = 'target'

### Read data

In [3]:
df = pd.read_csv('dataset/train.csv', dtype=object)

In [4]:
df.columns

Index(['building_id', 'building_material', 'city', 'txn_dt', 'total_floor',
       'building_type', 'building_use', 'building_complete_dt', 'parking_way',
       'parking_area',
       ...
       'XIV_500', 'XIV_index_500', 'XIV_1000', 'XIV_index_1000', 'XIV_5000',
       'XIV_index_5000', 'XIV_10000', 'XIV_index_10000', 'XIV_MIN',
       'total_price'],
      dtype='object', length=235)

### Preprocessing

In [5]:
# Convert types
df = convert_types(df, cols_num, col_target=col_target)

# Generate feats (train-test-same feats)
#create_feats(df)

### Feat engineering

In [6]:
from feature_engineering import CategoricalColumnsEncoder

In [11]:
class FeatureEngineering():
    def __init__(self):
        pass
    
    def fit_transform(self, df, cols_cat, cols_bin):
        df = df.copy()
        # Denote categorical-type
        self.cat_encoder = CategoricalColumnsEncoder(mode='pandas')
        self.cat_encoder.fit_transform(df, cols_cat+cols_bin)
        return df
    
    def transform(self, df):
        df = df.copy()
        self.cat_encoder.transform(df)
        return df

### Grid search

In [9]:
df_train = df.loc[itrain]

In [10]:
df_train.head()

,building_id,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,parking_area,...,XIV_index_500,XIV_1000,XIV_index_1000,XIV_5000,XIV_index_5000,XIV_10000,XIV_index_10000,XIV_MIN,total_price,target
20000,J85ey10qLrZniypR,8,12,20320.0,15.0,0,2,18112.0,0,NaN,...,1,73.0,1,4530.0,1,21112.0,1,77.962708,3.078642e+07,17.242584
20001,o0imPsqRzAQc0NWO,8,3,20325.0,5.0,1,2,8159.0,2,NaN,...,1,60.0,1,1454.0,1,2677.0,1,167.530197,8.490774e+05,13.651907
20002,r2NnNRI5oFSRhZ77,1,13,18311.0,4.0,1,2,2619.0,2,NaN,...,1,134.0,1,7211.0,1,29672.0,1,106.212357,6.052646e+06,15.616006
20003,yx8Gc5namoDlWkAa,8,12,19713.0,20.0,0,2,12420.0,0,2.487288,...,1,101.0,1,4587.0,1,21194.0,1,150.318359,5.417460e+06,15.505137
20004,baGhT0Wlw8y4Pcm0,8,21,19115.0,4.0,4,2,15403.0,2,NaN,...,1,76.0,1,1657.0,1,6154.0,1,143.308014,3.900504e+06,15.176617


In [13]:
# pars
is_log1p = True # if train on log1p target
# pars

# grid search
params_gsearch = {'task': ['train'],'boosting_type': ['gbdt'], 'metric': ['mae'],
                  'objective': ['mse'], # 'mae', 'mape'
                  'num_leaves': [31, 63, 255],
                  'learning_rate': [0.01, 0.05, 0.1],
                  'feature_fraction': [0.4, 0.7, 1.0],
                  'min_data_in_leaf': [20, 100],
                  'lambda_l1': [0, 0.1, 1]}

params_names = params_gsearch.keys()

gsearch = {}
folds = KFold(n_splits=3)
for i_fold, (itrain, ival) in enumerate(folds.split(df)): # kfold
    print('==== Fold', i_fold+1, '====')
    
    # split train, val
    df_train = df.loc[itrain]
    df_val = df.loc[ival]
    
    # feat eng
    feat_eng = FeatureEngineering()
    df_train = feat_eng.fit_transform(df_train, cols_cat, cols_bin)
    df_val = feat_eng.transform(df_val)

    # Construct lgb dataset
    if is_log1p:
        lgb_train = lgb.Dataset(df_train[cols_feats], df_train['target']).construct()
        lgb_val = lgb.Dataset(df_val[cols_feats], df_val['target'], reference=lgb_train).construct()
    else:
        lgb_train = lgb.Dataset(df_train[cols_feats], df_train['total_price']).construct()
        lgb_val = lgb.Dataset(df_val[cols_feats], df_val['total_price'], reference=lgb_train).construct()
        
    # grid search
    for values in itertools.product(*[params_gsearch[key] for key in params_names]):
        params = dict(zip(params_names, values))
        print('params:', params)
        
        model = lgb.train(params, lgb_train,
                          num_boost_round=10000,
                          valid_sets=lgb_val,
                          verbose_eval=200,
                          early_stopping_rounds=200)
        y_pred = model.predict(df_val[cols_feats])
        
        if is_log1p:
            y_pred_expm1 = np.expm1(y_pred)
            y_pred_final = np.clip(y_pred_expm1, 0, None)
        else:
            y_pred_final = y_pred
            
        score = cal_score(df_val['total_price'], y_pred_final)
        tuple_params = tuple(params.items())
        gsearch[tuple_params] = gsearch.get(tuple_params, []) + [score]

# aggregate, sort gsearch results
gsearch_results = [[key, np.mean(val), val] for key, val in gsearch.items()]
gsearch_results.sort(key= lambda x: x[1])
display(gsearch_results)

==== Fold 1 ====
params: {'task': 'train', 'boosting_type': 'gbdt', 'metric': 'mae', 'objective': 'mse', 'num_leaves': 31, 'learning_rate': 0.01, 'feature_fraction': 0.4, 'min_data_in_leaf': 20, 'lambda_l1': 0}
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's l1: 0.323333
[400]	valid_0's l1: 0.214895
[600]	valid_0's l1: 0.185709
[800]	valid_0's l1: 0.173781
[1000]	valid_0's l1: 0.167318
[1200]	valid_0's l1: 0.163388
[1400]	valid_0's l1: 0.160562
[1600]	valid_0's l1: 0.15835
[1800]	valid_0's l1: 0.156685
[2000]	valid_0's l1: 0.155332
[2200]	valid_0's l1: 0.154187
[2400]	valid_0's l1: 0.153157
[2600]	valid_0's l1: 0.152264
[2800]	valid_0's l1: 0.151463
[3000]	valid_0's l1: 0.150763
[3200]	valid_0's l1: 0.150115
[3400]	valid_0's l1: 0.149488
[3600]	valid_0's l1: 0.148886
[3800]	valid_0's l1: 0.148313
[4000]	valid_0's l1: 0.147808
[4200]	valid_0's l1: 0.147291
[4400]	valid_0's l1: 0.146847
[4600]	valid_0's l1: 0.146417
[4800]	valid_0's l1: 0.14604
[5000]	valid

[4400]	valid_0's l1: 0.147618
[4600]	valid_0's l1: 0.147201
[4800]	valid_0's l1: 0.146831
[5000]	valid_0's l1: 0.146488
[5200]	valid_0's l1: 0.14617
[5400]	valid_0's l1: 0.145873
[5600]	valid_0's l1: 0.145547
[5800]	valid_0's l1: 0.14526
[6000]	valid_0's l1: 0.144987
[6200]	valid_0's l1: 0.144747
[6400]	valid_0's l1: 0.144508
[6600]	valid_0's l1: 0.144274
[6800]	valid_0's l1: 0.144043
[7000]	valid_0's l1: 0.143777
[7200]	valid_0's l1: 0.143579
[7400]	valid_0's l1: 0.143388
[7600]	valid_0's l1: 0.14319
[7800]	valid_0's l1: 0.143009
[8000]	valid_0's l1: 0.142809
[8200]	valid_0's l1: 0.142629
[8400]	valid_0's l1: 0.142483
[8600]	valid_0's l1: 0.142308
[8800]	valid_0's l1: 0.14211
[9000]	valid_0's l1: 0.141949
[9200]	valid_0's l1: 0.141819
[9400]	valid_0's l1: 0.141667
[9600]	valid_0's l1: 0.141512
[9800]	valid_0's l1: 0.141376
[10000]	valid_0's l1: 0.141224
Did not meet early stopping. Best iteration is:
[10000]	valid_0's l1: 0.141224
params: {'task': 'train', 'boosting_type': 'gbdt', 'me

params: {'task': 'train', 'boosting_type': 'gbdt', 'metric': 'mae', 'objective': 'mse', 'num_leaves': 31, 'learning_rate': 0.01, 'feature_fraction': 0.7, 'min_data_in_leaf': 100, 'lambda_l1': 0}
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's l1: 0.29741
[400]	valid_0's l1: 0.201664
[600]	valid_0's l1: 0.179664
[800]	valid_0's l1: 0.170631
[1000]	valid_0's l1: 0.165498
[1200]	valid_0's l1: 0.162258
[1400]	valid_0's l1: 0.160032
[1600]	valid_0's l1: 0.15838
[1800]	valid_0's l1: 0.157034
[2000]	valid_0's l1: 0.155901
[2200]	valid_0's l1: 0.154922
[2400]	valid_0's l1: 0.15404
[2600]	valid_0's l1: 0.153223
[2800]	valid_0's l1: 0.152484
[3000]	valid_0's l1: 0.151841
[3200]	valid_0's l1: 0.151226
[3400]	valid_0's l1: 0.150687
[3600]	valid_0's l1: 0.150167
[3800]	valid_0's l1: 0.149675
[4000]	valid_0's l1: 0.149233
[4200]	valid_0's l1: 0.148848
[4400]	valid_0's l1: 0.14848
[4600]	valid_0's l1: 0.1481
[4800]	valid_0's l1: 0.147741
[5000]	valid_0's l1: 0.147425
[5

[4400]	valid_0's l1: 0.149624
[4600]	valid_0's l1: 0.149234
[4800]	valid_0's l1: 0.148851
[5000]	valid_0's l1: 0.148508
[5200]	valid_0's l1: 0.148177
[5400]	valid_0's l1: 0.147869
[5600]	valid_0's l1: 0.147582
[5800]	valid_0's l1: 0.147319
[6000]	valid_0's l1: 0.147042
[6200]	valid_0's l1: 0.146799
[6400]	valid_0's l1: 0.146572
[6600]	valid_0's l1: 0.146342
[6800]	valid_0's l1: 0.14612
[7000]	valid_0's l1: 0.145893
[7200]	valid_0's l1: 0.145669
[7400]	valid_0's l1: 0.145479
[7600]	valid_0's l1: 0.145285
[7800]	valid_0's l1: 0.145106
[8000]	valid_0's l1: 0.144928
[8200]	valid_0's l1: 0.144759
[8400]	valid_0's l1: 0.144603
[8600]	valid_0's l1: 0.144457
[8800]	valid_0's l1: 0.144315
[9000]	valid_0's l1: 0.144179
[9200]	valid_0's l1: 0.144043
[9400]	valid_0's l1: 0.143892
[9600]	valid_0's l1: 0.143745
[9800]	valid_0's l1: 0.143624
[10000]	valid_0's l1: 0.143504
Did not meet early stopping. Best iteration is:
[10000]	valid_0's l1: 0.143504
params: {'task': 'train', 'boosting_type': 'gbdt', 

params: {'task': 'train', 'boosting_type': 'gbdt', 'metric': 'mae', 'objective': 'mse', 'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.4, 'min_data_in_leaf': 20, 'lambda_l1': 0}
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's l1: 0.171399
[400]	valid_0's l1: 0.159224
[600]	valid_0's l1: 0.154568
[800]	valid_0's l1: 0.151715
[1000]	valid_0's l1: 0.149616
[1200]	valid_0's l1: 0.148
[1400]	valid_0's l1: 0.146641
[1600]	valid_0's l1: 0.145458
[1800]	valid_0's l1: 0.144509
[2000]	valid_0's l1: 0.143709
[2200]	valid_0's l1: 0.142999
[2400]	valid_0's l1: 0.142326
[2600]	valid_0's l1: 0.141756
[2800]	valid_0's l1: 0.141289
[3000]	valid_0's l1: 0.140982
[3200]	valid_0's l1: 0.140615
[3400]	valid_0's l1: 0.140316
[3600]	valid_0's l1: 0.140065
[3800]	valid_0's l1: 0.13981
[4000]	valid_0's l1: 0.139564
[4200]	valid_0's l1: 0.139372
[4400]	valid_0's l1: 0.139207
[4600]	valid_0's l1: 0.139074
[4800]	valid_0's l1: 0.138886
[5000]	valid_0's l1: 0.138739
[

...

[2600]	valid_0's l1: 0.138842
[2800]	valid_0's l1: 0.13882
[3000]	valid_0's l1: 0.138802
[3200]	valid_0's l1: 0.138786
[3400]	valid_0's l1: 0.138767
[3600]	valid_0's l1: 0.138766
[3800]	valid_0's l1: 0.138758
[4000]	valid_0's l1: 0.138747
[4200]	valid_0's l1: 0.13874
[4400]	valid_0's l1: 0.138728
[4600]	valid_0's l1: 0.138722
[4800]	valid_0's l1: 0.138718
[5000]	valid_0's l1: 0.138715
[5200]	valid_0's l1: 0.138708
[5400]	valid_0's l1: 0.138706
Early stopping, best iteration is:
[5376]	valid_0's l1: 0.138704
params: {'task': 'train', 'boosting_type': 'gbdt', 'metric': 'mae', 'objective': 'mse', 'num_leaves': 255, 'learning_rate': 0.05, 'feature_fraction': 0.4, 'min_data_in_leaf': 100, 'lambda_l1': 1}
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's l1: 0.154506
[400]	valid_0's l1: 0.145535
[600]	valid_0's l1: 0.142509
[800]	valid_0's l1: 0.141039
[1000]	valid_0's l1: 0.140351
[1200]	valid_0's l1: 0.139876
[1400]	valid_0's l1: 0.139589
[1600]	valid_0's l1: 0

[5400]	valid_0's l1: 0.139516
[5600]	valid_0's l1: 0.139515
[5800]	valid_0's l1: 0.139514
[6000]	valid_0's l1: 0.139513
[6200]	valid_0's l1: 0.139512
[6400]	valid_0's l1: 0.139512
[6600]	valid_0's l1: 0.139511
[6800]	valid_0's l1: 0.13951
[7000]	valid_0's l1: 0.13951
[7200]	valid_0's l1: 0.139509
[7400]	valid_0's l1: 0.139509
[7600]	valid_0's l1: 0.139508
[7800]	valid_0's l1: 0.139508
[8000]	valid_0's l1: 0.139508
[8200]	valid_0's l1: 0.139508
[8400]	valid_0's l1: 0.139507
[8600]	valid_0's l1: 0.139507
[8800]	valid_0's l1: 0.139507
[9000]	valid_0's l1: 0.139507
[9200]	valid_0's l1: 0.139506
[9400]	valid_0's l1: 0.139506
[9600]	valid_0's l1: 0.139506
[9800]	valid_0's l1: 0.139506
[10000]	valid_0's l1: 0.139506
Did not meet early stopping. Best iteration is:
[9997]	valid_0's l1: 0.139506
params: {'task': 'train', 'boosting_type': 'gbdt', 'metric': 'mae', 'objective': 'mse', 'num_leaves': 255, 'learning_rate': 0.05, 'feature_fraction': 1.0, 'min_data_in_leaf': 20, 'lambda_l1': 0.1}
Traini

[200]	valid_0's l1: 0.150461
[400]	valid_0's l1: 0.146401
[600]	valid_0's l1: 0.145287
[800]	valid_0's l1: 0.14477
[1000]	valid_0's l1: 0.144601
[1200]	valid_0's l1: 0.144509
Early stopping, best iteration is:
[1174]	valid_0's l1: 0.144509
params: {'task': 'train', 'boosting_type': 'gbdt', 'metric': 'mae', 'objective': 'mse', 'num_leaves': 255, 'learning_rate': 0.1, 'feature_fraction': 0.4, 'min_data_in_leaf': 100, 'lambda_l1': 0}
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's l1: 0.150012
[400]	valid_0's l1: 0.145688
[600]	valid_0's l1: 0.144606
[800]	valid_0's l1: 0.144227
[1000]	valid_0's l1: 0.144005
[1200]	valid_0's l1: 0.143923
[1400]	valid_0's l1: 0.143862
[1600]	valid_0's l1: 0.143842
[1800]	valid_0's l1: 0.143809
[2000]	valid_0's l1: 0.143794
[2200]	valid_0's l1: 0.143782
[2400]	valid_0's l1: 0.143768
[2600]	valid_0's l1: 0.143763
[2800]	valid_0's l1: 0.14376
[3000]	valid_0's l1: 0.143755
[3200]	valid_0's l1: 0.143754
Early stopping, best iterat

[5600]	valid_0's l1: 0.143251
[5800]	valid_0's l1: 0.143251
[6000]	valid_0's l1: 0.143251
Early stopping, best iteration is:
[5809]	valid_0's l1: 0.143251
params: {'task': 'train', 'boosting_type': 'gbdt', 'metric': 'mae', 'objective': 'mse', 'num_leaves': 255, 'learning_rate': 0.1, 'feature_fraction': 1.0, 'min_data_in_leaf': 20, 'lambda_l1': 0.1}
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's l1: 0.146007
[400]	valid_0's l1: 0.143988
[600]	valid_0's l1: 0.143403
[800]	valid_0's l1: 0.143134
[1000]	valid_0's l1: 0.143015
[1200]	valid_0's l1: 0.14295
[1400]	valid_0's l1: 0.142912
[1600]	valid_0's l1: 0.142893
[1800]	valid_0's l1: 0.142881
[2000]	valid_0's l1: 0.14288
Early stopping, best iteration is:
[1813]	valid_0's l1: 0.14288
params: {'task': 'train', 'boosting_type': 'gbdt', 'metric': 'mae', 'objective': 'mse', 'num_leaves': 255, 'learning_rate': 0.1, 'feature_fraction': 1.0, 'min_data_in_leaf': 20, 'lambda_l1': 1}
Training until validation scores d

[[(('task', 'train'),
   ('boosting_type', 'gbdt'),
   ('metric', 'mae'),
   ('objective', 'mse'),
   ('num_leaves', 31),
   ('learning_rate', 0.01),
   ('feature_fraction', 1.0),
   ('min_data_in_leaf', 100),
   ('lambda_l1', 1)),
  5023.182736892787,
  [5014.8459736956465, 5000.84919448768, 5053.853042495033]],
 [(('task', 'train'),
   ('boosting_type', 'gbdt'),
   ('metric', 'mae'),
   ('objective', 'mse'),
   ('num_leaves', 31),
   ('learning_rate', 0.01),
   ('feature_fraction', 1.0),
   ('min_data_in_leaf', 20),
   ('lambda_l1', 1)),
  5026.183789802272,
  [5038.847251680061, 5016.850701760834, 5022.853415965921]],
 [(('task', 'train'),
   ('boosting_type', 'gbdt'),
   ('metric', 'mae'),
   ('objective', 'mse'),
   ('num_leaves', 31),
   ('learning_rate', 0.01),
   ('feature_fraction', 1.0),
   ('min_data_in_leaf', 100),
   ('lambda_l1', 0.1)),
  5028.84949301197,
  [5020.8461883311675, 5016.849252355429, 5048.853038349313]],
 [(('task', 'train'),
   ('boosting_type', 'gbdt'),
  

In [14]:
display(gsearch_results)

[[(('task', 'train'),
   ('boosting_type', 'gbdt'),
   ('metric', 'mae'),
   ('objective', 'mse'),
   ('num_leaves', 31),
   ('learning_rate', 0.01),
   ('feature_fraction', 1.0),
   ('min_data_in_leaf', 100),
   ('lambda_l1', 1)),
  5023.182736892787,
  [5014.8459736956465, 5000.84919448768, 5053.853042495033]],
 [(('task', 'train'),
   ('boosting_type', 'gbdt'),
   ('metric', 'mae'),
   ('objective', 'mse'),
   ('num_leaves', 31),
   ('learning_rate', 0.01),
   ('feature_fraction', 1.0),
   ('min_data_in_leaf', 20),
   ('lambda_l1', 1)),
  5026.183789802272,
  [5038.847251680061, 5016.850701760834, 5022.853415965921]],
 [(('task', 'train'),
   ('boosting_type', 'gbdt'),
   ('metric', 'mae'),
   ('objective', 'mse'),
   ('num_leaves', 31),
   ('learning_rate', 0.01),
   ('feature_fraction', 1.0),
   ('min_data_in_leaf', 100),
   ('lambda_l1', 0.1)),
  5028.84949301197,
  [5020.8461883311675, 5016.849252355429, 5048.853038349313]],
 [(('task', 'train'),
   ('boosting_type', 'gbdt'),
  